<a href="https://colab.research.google.com/github/patrickbryant1/MoLPC/blob/master/MoLPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install dependencies
%pip install biopython==1.79
%pip install JAX==0.2.14
%pip install dm-haiku==0.0.4
%pip install ml-collections==0.1.0
%pip install chex==0.0.7
%pip install dm-tree==0.1.6
%pip install immutabledict==2.0.0
%pip install numpy==1.19.5
%pip install pandas==1.3.4
%pip install scipy==1.7.0
%pip install tensorflow-cpu==2.5.0

In [ ]:
#@title Clone MoLPC git
import shutil
shutil.rmtree('/content/MoLPC', ignore_errors=True)
!git clone https://github.com/patrickbryant1/MoLPC.git

In [ ]:
#@title #Run the assembly pipeline
import sys, os
from google.colab import files
sys.path.insert(0,'/content/MoLPC/src')
#@title Default title text
ID = "1A8R" #@param {type:"string"}
SUBSIZE = 3 #@param {type:"integer"} #What order the subcomplexes should be (2 or 3)
GET_ALL = 1 #@param {type:"integer"} #If to get all interactions (1) or not (0) - when the interactions are known
#Get the csvs
import pandas as pd
import numpy as np
DATADIR='/content/MoLPC/data/test/'
USEQS=pd.read_csv(DATADIR+ID+'_useqs.csv')
CHAINS=pd.read_csv(DATADIR+ID+'_chains.csv')
INTERACTIONS='' #Leave empty if the interactions are not known - here they are not used. See the file $DATADIR/$ID'_ints.csv' to how to supply such a file
MSADIR=DATADIR+'/'
OUTDIR="/content/gdrive/MyDrive/"
#Mount the drive to be able to save files
from google.colab import drive
drive.mount('/content/gdrive') #All the output will be written here
#@markdown You have to allow to connect to Google drive in order to run MoLPC.

In [ ]:
#@title Step 1: MSA PIPELINE
#@markdown Now, a default MSA is read in - no search is performed here

#@markdown Write the Paired and Block Diagonalized MSAs to predict sub-components
from preprocess import preprocess_colab
preprocess_colab.create_folder_structure(MSADIR, ID, OUTDIR, USEQS, INTERACTIONS, CHAINS, GET_ALL, SUBSIZE)

In [4]:
#@title Step 2: FOLDING PIPELINE
#Create structure dir
STRUCTURE_DIR=OUTDIR+"AF/"
if not os.path.exists(STRUCTURE_DIR):
  os.mkdir(STRUCTURE_DIR)
#Get the sub_ids and lengths
import glob
files = glob.glob(OUTDIR+'*.fasta')
sub_ids = {}
for filename in files:
  with open(filename, 'r') as file:
    for line in file:
      line = line.rstrip()[1:].split('|')
      sub_ids[line[0]]=line[-1].split('-')[:-1]
      break

#@markdown Get the AF2 params
import shutil
PARAMS=STRUCTURE_DIR+'params/'
if not os.path.exists(PARAMS):
  os.mkdir(PARAMS)
  !wget https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar 
  shutil.move('/content/alphafold_params_2021-07-14.tar', PARAMS)
  #Extract
  !tar -xvf /content/gdrive/MyDrive/AF/params/alphafold_params_2021-07-14.tar -C /content/gdrive/MyDrive/AF/params/

In [ ]:
#@markdown Predict the subcomponents
sys.path.insert(0,'/content/MoLPC/src/AF2')
from AF2 import run_alphafold_colab
##### AF2 CONFIGURATION #####
PARAM=STRUCTURE_DIR
PRESET='full_dbs' #Choose preset model configuration - no ensembling (full_dbs) and (reduced_dbs) or 8 model ensemblings (casp14).
MAX_RECYCLES=10 #max_recycles (default=3)
MODEL_NAME='model_1' #model_1_ptm

#Go through all subcomponents and predict their structure
for sub_id in sub_ids:
  ####Get fasta file####
  FASTAFILE=OUTDIR+sub_id+'.fasta'
  ####Get chain break#### Note! This is now set for trimer subcomponents
  CB=np.cumsum([int(x) for x in sub_ids[sub_id]])
  CB = [str(x) for x in CB]
  ####Get MSAs####
  #HHblits paired
  PAIREDMSA=OUTDIR+sub_id+'_paired.a3m'
  ##HHblits block diagonalized
  BLOCKEDMSA=OUTDIR+sub_id+'_blocked.a3m'
  MSAS=[PAIREDMSA,BLOCKEDMSA] #Comma separated list of msa paths
  run_alphafold_colab.main([MODEL_NAME], 1, MAX_RECYCLES, STRUCTURE_DIR, FASTAFILE, sub_id, MSAS, CB, OUTDIR)

In [ ]:
#@title Step 3: ASSEMBLY PIPELINE
#@markdown Prepare the assembly
COMPLEXDIR=OUTDIR+'/assembly/complex/' #Where all the output for the complex assembly will be directed
PAIRDIR=OUTDIR+'/assembly/pairs/'
META=OUTDIR+'/assembly/meta.csv' #where to write all interactions
from complex_assembly import prepare_assembly_colab
#Make complex directory
if not os.path.exists(COMPLEXDIR):
  os.mkdir(OUTDIR+'/assembly')
  os.mkdir(COMPLEXDIR)
#Rewrite the FoldDock preds to have separate chains according to the fasta file seqlens
files = glob.glob(OUTDIR+ID+'*/*1.pdb')
if len(files)>0:
    for pdbname in files:
        chains = prepare_assembly_colab.read_all_chains_coords(pdbname)
        if len(chains.keys())>1:
            continue
        subid = pdbname.split('/')[-2]
        print(subid)
        #Rewrite the files
        prepare_assembly_colab.write_pdb(chains, pdbname.split('.')[0]+'_rw'+'.pdb')

#Copy the predictions to reflect all chains
prepare_assembly_colab.copy_uints(ID, OUTDIR, OUTDIR+'/assembly/', USEQS,INTERACTIONS, CHAINS, GET_ALL, SUBSIZE)
##Rewrite AF predicted complexes to have proper numbering and chain labels
files = glob.glob(OUTDIR+'/assembly/'+ID+'*/*.pdb')
if len(files)>0:
    for pdbname in files:
        chains = prepare_assembly_colab.read_all_chains_coords(pdbname)
        subid = pdbname.split('/')[-2]
        chain_names = subid.split('_')[-1]
        #Rewrite the files
        prepare_assembly_colab.write_pdb_chain_labels(chains, chain_names, OUTDIR+'/assembly/'+subid+'.pdb')
#Write all pairs
#It is necessary that the first unique chain is named A-..N for and the second N-... and so on
if not os.path.exists(PAIRDIR):
  os.mkdir(PAIRDIR)

prepare_assembly_colab.get_all_pairs(OUTDIR+'/assembly/', PAIRDIR, INTERACTIONS, GET_ALL, META)
#Cleanup
for filename in glob.glob(OUTDIR+'/assembly/'+ID+'_*.pdb'):
  os.remove(filename)
for dir in glob.glob(OUTDIR+'/assembly/'+ID+'_*'):
  if os.path.isdir(dir)==True:
    shutil.rmtree(dir)


In [ ]:
#@markdown Assemble: find the best non-overlapping path that connect all nodes using Monte Carlo tree search
META_DF=pd.read_csv(META)
CHAIN_SEQS=pd.read_csv(OUTDIR+'/assembly/'+ID+'_chains.csv')
from complex_assembly import mcts_colab
mcts_colab.assemble(META_DF, PAIRDIR, OUTDIR+'/assembly/plddt/', USEQS, CHAIN_SEQS, COMPLEXDIR)
